## Задача №1

Напишите класс, который реализует Стек и его возможности (достаточно будет добавления и удаления элемента).

После этого напишите ещё один класс “Менеджер задач”. В менеджере задач можно выполнить команду “новая задача”, в которую передаётся сама задача (str) и её приоритет (int). Сам менеджер работает на основе Стэка.  При выводе менеджера в консоль все задачи должны быть отсортированы по приоритету: чем меньше число, тем выше задача.

Дополнительно: реализуйте также удаление задач и подумайте, что делать с дубликатами

In [2]:
class Stack:
    def __init__(self):
        self.items = []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        if not self.is_empty():
            return self.items.pop()
        return None

    def is_empty(self):
        return len(self.items) == 0

    def peek(self):
        if not self.is_empty():
            return self.items[-1]
        return None

    def size(self):
        return len(self.items)


class TaskManager:
    def __init__(self):
        self.stack = Stack()
        self.tasks = {}  # Словарь для хранения задач и их приоритетов

    def new_task(self, task: str, priority: int):
        if task in self.tasks:
            # Если задача уже существует, обновляем приоритет
            self.tasks[task] = min(self.tasks[task], priority)
        else:
            self.tasks[task] = priority
        self.stack.push((priority, task))

    def remove_task(self, task: str):
        if task in self.tasks:
            del self.tasks[task]
            # Обновляем стек, чтобы удалить задачу
            new_stack = Stack()
            while not self.stack.is_empty():
                priority, t = self.stack.pop()
                if t != task:
                    new_stack.push((priority, t))
            self.stack = new_stack

    def __str__(self):
        tasks = list(self.tasks.items())
        tasks.sort(key=lambda x: x[1])  # Сотрируем по приоритету

        # Формируем строку для вывода
        result = []
        current_priority = None
        current_tasks = []

        for task, priority in tasks:
            if current_priority is None or current_priority != priority:
                if current_tasks:
                    result.append(f"{current_priority} {'; '.join(current_tasks)}")
                current_priority = priority
                current_tasks = [task]
            else:
                current_tasks.append(task)

        if current_tasks:
            result.append(f"{current_priority} {'; '.join(current_tasks)}")

        return "\n".join(result)


# Пример
manager = TaskManager()
manager.new_task("сделать уборку", 4)
manager.new_task("помыть посуду", 4)
manager.new_task("отдохнуть", 1)
manager.new_task("поесть", 2)
manager.new_task("сдать дз", 2)

print("Задачи до удаления:")
print(manager)

manager.remove_task("поесть")

print("\nЗадачи после удаления 'поесть':")
print(manager)

Задачи до удаления:
1 отдохнуть
2 поесть; сдать дз
4 сделать уборку; помыть посуду

Задачи после удаления 'поесть':
1 отдохнуть
2 сдать дз
4 сделать уборку; помыть посуду


## Задача №2

Вы разрабатываете программу для кэширования запросов к внешнему API. 

1. Создайте класс LRU Cache, который хранит ограниченное количество объектов и, при превышении лимита, удаляет самые давние (самые старые) использованные элементы.
2. Реализуйте методы добавления и извлечения элементов с использованием декораторов property и setter.

In [6]:
class LRUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.cache = {}
        self.order = []

    @property
    def oldest(self):
        """Возвращает самый старый элемент в кэше."""
        if self.order:
            return self.order[0]
        return None

    @oldest.setter
    def oldest(self, new_elem):
        """Добавляет новый элемент в кэш."""
        key, value = new_elem
        if key in self.cache:
            # Если элемент уже существует, обновляем его и перемещаем в конец
            self.order.remove(key)
            self.order.append(key)
        else:
            # Если элемент новый, добавляем его в кэш
            if len(self.cache) >= self.capacity:
                oldest_key = self.order.pop(0)  # Удаляем самый старый элемент
                del self.cache[oldest_key]
            self.cache[key] = value  # Сохраняем значение
            self.order.append(key)

    def get(self, key):
        """Извлекает элемент из кэша, если он существует."""
        if key in self.cache:
            # Перемещаем элемент в конец, так как он был использован
            self.order.remove(key)
            self.order.append(key)
            return self.cache[key]
        return None

    def print_cache(self):
        """Выводит текущий кэш."""
        print("LRU Cache:")
        for key in self.order:
            print(f"{key} : {self.cache[key]}")
        print('\n') #Дял разделения выводов


# Пример использования
cache = LRUCache(3)

# Добавляем элементы в кэш
cache.oldest = ("key1", "value1")
cache.oldest = ("key2", "value2")
cache.oldest = ("key3", "value3")

# Выводим текущий кэш
cache.print_cache()  # key1 : value1, key2 : value2, key3 : value3

# Получаем значение по ключу
print(cache.get("key2"))  # value2

# Добавляем новый элемент, превышающий лимит capacity
cache.oldest = ("key4", "value4")

# Выводим обновлённый кэш
cache.print_cache()  # key2 : value2, key3 : value3, key4 : value4

LRU Cache:
key1 : value1
key2 : value2
key3 : value3


value2
LRU Cache:
key3 : value3
key2 : value2
key4 : value4




## Задача 3

Создайте декоратор, который кэширует (сохраняет для дальнейшего использования) результаты вызова функции и, при повторном вызове с теми же аргументами, возвращает сохранённый результат.

Примените его к рекурсивной функции вычисления чисел Фибоначчи.
В итоге декоратор должен проверять аргументы, с которыми вызывается функция, и, если такие аргументы уже использовались, должен вернуть сохранённый результат вместо запуска расчёта.
Советы:
- Для хранения результатов удобно использовать словарь, так как поиск элементов внутри словаря будет иметь сложность, равную в среднем O(1).
- При этом не стоит хранить все вычисления в одном словаре, созданном снаружи функций (в глобальной области видимости). Лучше создавать отдельные словари для каждой декорируемой функции.

In [9]:
def cache_results(func):
    cache = {}  # Словарь для хранения результатов

    def wrapper(*args):
        if args in cache:
            return cache[args]  # Возвращаем сохранённый результат
        result = func(*args)  # Вызываем оригинальную функцию
        cache[args] = result  # Сохраняем результат в кэше
        return result

    return wrapper

@cache_results
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

# Примеры использования
print(fibonacci(10))
print(fibonacci(20))
print(fibonacci(30))
print(fibonacci(10))

55
6765
832040
55


## Задача №4

Напишите программу, которая реализует игру Крестики-нолики. Ваши классы в этой задаче могут выглядеть так:
```python
class Cell:
   #  Клетка, у которой есть значения
   #   - занята она или нет
   #   - номер клетки

class Board:
   #  Класс поля, который создаёт у себя экземпляры клетки

class Player:
   #  У игрока может быть
   #   - имя
   #   - на какую клетку ходит
```
*Пояснение к решению:*
1. **Класс Cell**: Представляет клетку на поле. У нее есть номер, статус занятости и информация о том, какой игрок занял клетку.
2. **Класс Board**: Создает поле из 9 клеток и содержит методы для отображения поля, проверки победителя и проверки заполненности поля.
3. **Класс Player**: Представляет игрока с именем и методом для совершения хода.
4. **Функция play_game**: Основная логика игры, которая управляет ходами игроков, проверяет условия победы и завершения игры.

In [10]:
class Cell:
    def __init__(self, number):
        self.number = number
        self.occupied = False
        self.player = None

    def occupy(self, player):
        if not self.occupied:
            self.occupied = True
            self.player = player
            return True
        return False

    def __str__(self):
        return self.player if self.occupied else str(self.number)


class Board:
    def __init__(self):
        self.cells = [Cell(i) for i in range(1, 10)]  # Создаем 9 клеток

    def display(self):
        for i in range(3):
            print(" | ".join(str(self.cells[j]) for j in range(i * 3, (i + 1) * 3)))
            if i < 2:
                print("---------")

    def is_winner(self, player): # Проверка возможных выигрышных комбинаций
        winning_combinations = [
            [0, 1, 2],  # строки
            [3, 4, 5],  
            [6, 7, 8],  
            [0, 3, 6],  # колонки
            [1, 4, 7],  
            [2, 5, 8],  
            [0, 4, 8],  # диагонали
            [2, 4, 6]   
        ]
        for combination in winning_combinations:
            if all(self.cells[i].player == player for i in combination):
                return True
        return False

    def is_full(self):
        return all(cell.occupied for cell in self.cells)


class Player:
    def __init__(self, name):
        self.name = name

    def make_move(self, board, cell_number):
        cell = board.cells[cell_number - 1]
        return cell.occupy(self.name)


def play_game():
    board = Board()
    player1 = Player("X")
    player2 = Player("O")
    current_player = player1

    while True:
        board.display()
        move = int(input(f"{current_player.name}, выберите клетку (1-9): "))
        if move < 1 or move > 9 or not current_player.make_move(board, move):
            print("Неверный ход, попробуйте снова.")
            continue

        if board.is_winner(current_player.name):
            board.display()
            print(f"Игрок {current_player.name} выиграл!")
            break

        if board.is_full():
            board.display()
            print("Игра закончилась вничью!")
            break

        current_player = player2 if current_player == player1 else player1


# Запуск игры
play_game()

1 | 2 | 3
---------
4 | 5 | 6
---------
7 | 8 | 9
X, выберите клетку (1-9): 1
X | 2 | 3
---------
4 | 5 | 6
---------
7 | 8 | 9
O, выберите клетку (1-9): 5
X | 2 | 3
---------
4 | O | 6
---------
7 | 8 | 9
X, выберите клетку (1-9): 2
X | X | 3
---------
4 | O | 6
---------
7 | 8 | 9
O, выберите клетку (1-9): 4
X | X | 3
---------
O | O | 6
---------
7 | 8 | 9
X, выберите клетку (1-9): 3
X | X | X
---------
O | O | 6
---------
7 | 8 | 9
Игрок X выиграл!
